In [2]:
pip install kaggle

^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
import os
import shutil

# Create the directory
os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)

# Copy the kaggle.json file to the .kaggle directory
shutil.copy('kaggle.json', os.path.expanduser('~/.kaggle/'))

# Set appropriate permissions for the kaggle.json file (not applicable in Windows but no harm to leave it)
os.chmod(os.path.expanduser('~/.kaggle/kaggle.json'), 0o600)

In [4]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other




  0%|          | 0.00/25.7M [00:00<?, ?B/s]
  4%|▍         | 1.00M/25.7M [00:01<00:42, 605kB/s]
  8%|▊         | 2.00M/25.7M [00:02<00:22, 1.13MB/s]
 12%|█▏        | 3.00M/25.7M [00:02<00:14, 1.64MB/s]
 16%|█▌        | 4.00M/25.7M [00:02<00:10, 2.22MB/s]
 19%|█▉        | 5.00M/25.7M [00:02<00:08, 2.58MB/s]
 23%|██▎       | 6.00M/25.7M [00:03<00:06, 3.04MB/s]
 27%|██▋       | 7.00M/25.7M [00:03<00:05, 3.56MB/s]
 31%|███       | 8.00M/25.7M [00:03<00:05, 3.54MB/s]
 35%|███▌      | 9.00M/25.7M [00:03<00:04, 3.91MB/s]
 39%|███▉      | 10.0M/25.7M [00:04<00:03, 4.27MB/s]
 43%|████▎     | 11.0M/25.7M [00:04<00:03, 4.45MB/s]
 47%|████▋     | 12.0M/25.7M [00:04<00:03, 4.61MB/s]
 51%|█████     | 13.0M/25.7M [00:04<00:02, 4.94MB/s]
 54%|█████▍    | 14.0M/25.7M [00:04<00:02, 4.53MB/s]
 58%|█████▊    | 15.0M/25.7M [00:05<00:02, 4.72MB/s]
 62%|██████▏   | 16.0M/25.7M [00:05<00:02, 3.83MB/s]
 66%|██████▌   | 17.0M/25.7M [00:05<00:02, 4.17MB/s]
 70%|███████   | 18.0M/25.7M [00:05<00:02, 3.94MB/s]
 7

In [6]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 982B-74D1

 Directory of c:\Users\Uyama\Downloads\JANE-UDEMY\NLP\Deep-Learning-NLP

09/09/2024  07:25    <DIR>          .
26/08/2024  03:11    <DIR>          ..
28/08/2024  13:17    <DIR>          annclassNLP
19/10/2019  19:25        26,962,657 imdb-dataset-of-50k-movie-reviews.zip
08/09/2024  14:56             5,222 IMDB-reviews-Sentiment-Analysis.ipynb
01/09/2024  07:39                65 kaggle.json
05/09/2024  07:15    <DIR>          LSTM-RNN
19/08/2024  15:12               101 requirements.txt
02/09/2024  14:27    <DIR>          SimpleRNN
18/08/2024  09:28    <DIR>          venv
               4 File(s)     26,968,045 bytes
               6 Dir(s)  48,034,299,904 bytes free


In [7]:
# # Unzip the downloaded dataset
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", 'r') as zip_ref:
     zip_ref.extractall()

In [8]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 982B-74D1

 Directory of c:\Users\Uyama\Downloads\JANE-UDEMY\NLP\Deep-Learning-NLP

09/09/2024  07:26    <DIR>          .
26/08/2024  03:11    <DIR>          ..
28/08/2024  13:17    <DIR>          annclassNLP
09/09/2024  07:26        66,212,309 IMDB Dataset.csv
19/10/2019  19:25        26,962,657 imdb-dataset-of-50k-movie-reviews.zip
08/09/2024  14:56             5,222 IMDB-reviews-Sentiment-Analysis.ipynb
01/09/2024  07:39                65 kaggle.json
05/09/2024  07:15    <DIR>          LSTM-RNN
19/08/2024  15:12               101 requirements.txt
02/09/2024  14:27    <DIR>          SimpleRNN
18/08/2024  09:28    <DIR>          venv
               5 File(s)     93,180,354 bytes
               6 Dir(s)  47,969,062,912 bytes free


### Load the dataset

In [4]:
df = pd.read_csv("IMDB Dataset.csv")

In [5]:
df.shape

(50000, 2)

In [6]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
df.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [8]:
df["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [9]:
df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

C:\Users\Uyama\AppData\Local\Temp\ipykernel_20256\1137712857.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [10]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [11]:
df["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [12]:
X=df["review"]
y=df["sentiment"]

### Model Training

In [13]:
#split the data into training and test data
X_train, X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [14]:
print(X_train.shape)
print(X_test.shape)

(40000,)
(10000,)


In [15]:
print(df.shape)

(50000, 2)


In [16]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure equal length (maxlen=200)
X_train_padded = pad_sequences(X_train_seq, maxlen=200)
X_test_padded = pad_sequences(X_test_seq, maxlen=200)

print("X_train_padded shape:", X_train_padded.shape)


X_train_padded shape: (40000, 200)


In [17]:
print(X_train_padded)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [18]:
print(X_test_padded)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [19]:
print(y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [20]:
print(y_test)

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


### LTSM - Long Short-Term Memory

In [21]:
# build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          640000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [24]:
model.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5


500/500 [==============================] - 1017s 2s/step - loss: 0.4032 - accuracy: 0.8157 - val_loss: 0.2910 - val_accuracy: 0.8786
Epoch 2/5
500/500 [==============================] - 1026s 2s/step - loss: 0.2664 - accuracy: 0.8953 - val_loss: 0.3200 - val_accuracy: 0.8742
Epoch 3/5
500/500 [==============================] - 1003s 2s/step - loss: 0.2204 - accuracy: 0.9142 - val_loss: 0.3117 - val_accuracy: 0.8801
Epoch 4/5
500/500 [==============================] - 982s 2s/step - loss: 0.1891 - accuracy: 0.9266 - val_loss: 0.3459 - val_accuracy: 0.8739
Epoch 5/5
500/500 [==============================] - 1077s 2s/step - loss: 0.1609 - accuracy: 0.9394 - val_loss: 0.3491 - val_accuracy: 0.8751


In [25]:
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 [==============================] - 20s 62ms/step - loss: 0.3244 - accuracy: 0.8816
Test Loss: 0.3244113326072693
Test Accuracy: 0.881600022315979


In [26]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [27]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 1s 740ms/step
The sentiment of the review is: positive


In [28]:
# example usage
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 54ms/step
The sentiment of the review is: negative


In [29]:
# example usage
new_review = "This movie was ok but not that good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 51ms/step
The sentiment of the review is: negative
